This is going to be a basketball prediction machine learning model. Use the NBA stats and try to predict outcomes of Lakers games

# Initial prep for training dataset

In [2]:
import pandas as pd

In [3]:
from nba_api.stats.endpoints import leaguegamelog

# Create an instance of the LeagueGameLog endpoint
game_log = leaguegamelog.LeagueGameLog()

# Retrieve the data from the API
game_log_data = game_log.get_data_frames()[0]
game_data = pd.DataFrame(game_log_data)
lal_games = game_log_data[game_log_data['TEAM_ABBREVIATION'] == 'LAL']

In [4]:
lal_games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22023,1610612747,LAL,Los Angeles Lakers,0022300061,2023-10-24,LAL @ DEN,L,240,41,...,31,44,23,5,4,12,18,107,-12,1
30,22023,1610612747,LAL,Los Angeles Lakers,0022300076,2023-10-26,LAL vs. PHX,W,240,36,...,34,42,24,14,7,16,23,100,5,1
76,22023,1610612747,LAL,Los Angeles Lakers,0022300100,2023-10-29,LAL @ SAC,L,265,43,...,45,53,27,4,4,12,22,127,-5,1
81,22023,1610612747,LAL,Los Angeles Lakers,0022300111,2023-10-30,LAL vs. ORL,W,240,42,...,37,40,28,7,10,16,17,106,3,1
108,22023,1610612747,LAL,Los Angeles Lakers,0022300127,2023-11-01,LAL vs. LAC,W,265,46,...,42,51,28,10,9,14,24,130,5,1


In [5]:
lal_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, 0 to 1484
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SEASON_ID          52 non-null     object 
 1   TEAM_ID            52 non-null     int64  
 2   TEAM_ABBREVIATION  52 non-null     object 
 3   TEAM_NAME          52 non-null     object 
 4   GAME_ID            52 non-null     object 
 5   GAME_DATE          52 non-null     object 
 6   MATCHUP            52 non-null     object 
 7   WL                 52 non-null     object 
 8   MIN                52 non-null     int64  
 9   FGM                52 non-null     int64  
 10  FGA                52 non-null     int64  
 11  FG_PCT             52 non-null     float64
 12  FG3M               52 non-null     int64  
 13  FG3A               52 non-null     int64  
 14  FG3_PCT            52 non-null     float64
 15  FTM                52 non-null     int64  
 16  FTA                52 non-null 

In [6]:
lal_games = lal_games.drop(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MIN', 'PLUS_MINUS', 'VIDEO_AVAILABLE'], axis=1)

In [7]:
lal_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, 0 to 1484
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   MATCHUP  52 non-null     object 
 1   WL       52 non-null     object 
 2   FGM      52 non-null     int64  
 3   FGA      52 non-null     int64  
 4   FG_PCT   52 non-null     float64
 5   FG3M     52 non-null     int64  
 6   FG3A     52 non-null     int64  
 7   FG3_PCT  52 non-null     float64
 8   FTM      52 non-null     int64  
 9   FTA      52 non-null     int64  
 10  FT_PCT   52 non-null     float64
 11  OREB     52 non-null     int64  
 12  DREB     52 non-null     int64  
 13  REB      52 non-null     int64  
 14  AST      52 non-null     int64  
 15  STL      52 non-null     int64  
 16  BLK      52 non-null     int64  
 17  TOV      52 non-null     int64  
 18  PF       52 non-null     int64  
 19  PTS      52 non-null     int64  
dtypes: float64(3), int64(15), object(2)
memory usage: 8.5+ KB


In [8]:
lal_games['TEAM_ABBREVIATION'] = lal_games['MATCHUP'].str[-3:]
lal_games = lal_games.drop(['MATCHUP'], axis=1)
lal_games['TEAM_ABBREVIATION']

0       DEN
30      PHX
76      SAC
81      ORL
108     LAC
167     ORL
204     MIA
232     HOU
251     PHX
285     POR
296     MEM
326     SAC
338     POR
371     HOU
403     UTA
413     DAL
462     CLE
491     PHI
519     DET
541     OKC
561     HOU
589     PHX
612     NOP
677     DAL
684     SAS
720     SAS
775     NYK
795     CHI
809     MIN
848     OKC
864     BOS
915     CHA
941     MIN
957     NOP
1001    MIA
1039    MEM
1056    LAC
1090    TOR
1112    PHX
1156    UTA
1169    OKC
1197    DAL
1237    BKN
1255    POR
1291    LAC
1320    CHI
1338    GSW
1380    HOU
1394    ATL
1425    BOS
1453    NYK
1484    CHA
Name: TEAM_ABBREVIATION, dtype: object

In [9]:
lal_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, 0 to 1484
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   WL                 52 non-null     object 
 1   FGM                52 non-null     int64  
 2   FGA                52 non-null     int64  
 3   FG_PCT             52 non-null     float64
 4   FG3M               52 non-null     int64  
 5   FG3A               52 non-null     int64  
 6   FG3_PCT            52 non-null     float64
 7   FTM                52 non-null     int64  
 8   FTA                52 non-null     int64  
 9   FT_PCT             52 non-null     float64
 10  OREB               52 non-null     int64  
 11  DREB               52 non-null     int64  
 12  REB                52 non-null     int64  
 13  AST                52 non-null     int64  
 14  STL                52 non-null     int64  
 15  BLK                52 non-null     int64  
 16  TOV                52 non-null 

# Now we pull the team rankings by win_pct

In [11]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguestandingsv3

# Get all the teams' IDs
nba_teams = teams.get_teams()

# Select the desired season
current_season = '2023-24'

# Get the standings
standings = leaguestandingsv3.LeagueStandingsV3(season=current_season)
standings_df = standings.get_data_frames()[0]

standings_df = standings_df[['TeamID', 'TeamName', 'WinPCT']]
standings_df = standings_df.sort_values(by='WinPCT', ascending=False).reset_index(drop=True)
standings_df['Rank'] = standings_df.index + 1
print(standings_df)



        TeamID       TeamName  WinPCT  Rank
0   1610612738        Celtics   0.760     1
1   1610612760        Thunder   0.700     2
2   1610612750   Timberwolves   0.700     3
3   1610612746       Clippers   0.694     4
4   1610612743        Nuggets   0.686     5
5   1610612739      Cavaliers   0.667     6
6   1610612749          Bucks   0.660     7
7   1610612752         Knicks   0.640     8
8   1610612755          76ers   0.612     9
9   1610612758          Kings   0.592    10
10  1610612756           Suns   0.580    11
11  1610612740       Pelicans   0.580    12
12  1610612754         Pacers   0.549    13
13  1610612753          Magic   0.540    14
14  1610612742      Mavericks   0.540    15
15  1610612748           Heat   0.520    16
16  1610612747         Lakers   0.519    17
17  1610612762           Jazz   0.490    18
18  1610612745        Rockets   0.469    19
19  1610612744       Warriors   0.468    20
20  1610612741          Bulls   0.460    21
21  1610612737          Hawks   

Merge another table onto this to get the team abbreviation, then pull the abbreviation from lal_games then we can merge those

In [12]:
standings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TeamID    30 non-null     int64  
 1   TeamName  30 non-null     object 
 2   WinPCT    30 non-null     float64
 3   Rank      30 non-null     int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 1.1+ KB


In [13]:
mask = standings_df['TeamName'] == 'Lakers'

standings_df = standings_df[~mask]

print(standings_df)

        TeamID       TeamName  WinPCT  Rank
0   1610612738        Celtics   0.760     1
1   1610612760        Thunder   0.700     2
2   1610612750   Timberwolves   0.700     3
3   1610612746       Clippers   0.694     4
4   1610612743        Nuggets   0.686     5
5   1610612739      Cavaliers   0.667     6
6   1610612749          Bucks   0.660     7
7   1610612752         Knicks   0.640     8
8   1610612755          76ers   0.612     9
9   1610612758          Kings   0.592    10
10  1610612756           Suns   0.580    11
11  1610612740       Pelicans   0.580    12
12  1610612754         Pacers   0.549    13
13  1610612753          Magic   0.540    14
14  1610612742      Mavericks   0.540    15
15  1610612748           Heat   0.520    16
17  1610612762           Jazz   0.490    18
18  1610612745        Rockets   0.469    19
19  1610612744       Warriors   0.468    20
20  1610612741          Bulls   0.460    21
21  1610612737          Hawks   0.440    22
22  1610612751           Nets   

In [14]:
standings_df = standings_df.drop(['TeamName', 'WinPCT'], axis=1)

In [15]:
standings_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, 0 to 29
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   TeamID  29 non-null     int64
 1   Rank    29 non-null     int64
dtypes: int64(2)
memory usage: 696.0 bytes


# Pull Team ID and Team abbreviation

In [16]:
from nba_api.stats.static import teams

# Get all the teams' information
nba_teams = teams.get_teams()

# Create an empty list to store the team data
team_data = []

# Loop through each team and extract the Team_ID and Team_Abbreviation
for team in nba_teams:
    team_id = team['id']
    team_abbreviation = team['abbreviation']
    team_data.append({'Team_ID': team_id, 'Team_Abbreviation': team_abbreviation})

# Create a DataFrame from the team data
team_df = pd.DataFrame(team_data)

# Print the team DataFrame
print(team_df)

       Team_ID Team_Abbreviation
0   1610612737               ATL
1   1610612738               BOS
2   1610612739               CLE
3   1610612740               NOP
4   1610612741               CHI
5   1610612742               DAL
6   1610612743               DEN
7   1610612744               GSW
8   1610612745               HOU
9   1610612746               LAC
10  1610612747               LAL
11  1610612748               MIA
12  1610612749               MIL
13  1610612750               MIN
14  1610612751               BKN
15  1610612752               NYK
16  1610612753               ORL
17  1610612754               IND
18  1610612755               PHI
19  1610612756               PHX
20  1610612757               POR
21  1610612758               SAC
22  1610612759               SAS
23  1610612760               OKC
24  1610612761               TOR
25  1610612762               UTA
26  1610612763               MEM
27  1610612764               WAS
28  1610612765               DET
29  161061

In [17]:
mask = team_df['Team_Abbreviation'] == 'LAL'

team_df = team_df[~mask]

print(team_df)

       Team_ID Team_Abbreviation
0   1610612737               ATL
1   1610612738               BOS
2   1610612739               CLE
3   1610612740               NOP
4   1610612741               CHI
5   1610612742               DAL
6   1610612743               DEN
7   1610612744               GSW
8   1610612745               HOU
9   1610612746               LAC
11  1610612748               MIA
12  1610612749               MIL
13  1610612750               MIN
14  1610612751               BKN
15  1610612752               NYK
16  1610612753               ORL
17  1610612754               IND
18  1610612755               PHI
19  1610612756               PHX
20  1610612757               POR
21  1610612758               SAC
22  1610612759               SAS
23  1610612760               OKC
24  1610612761               TOR
25  1610612762               UTA
26  1610612763               MEM
27  1610612764               WAS
28  1610612765               DET
29  1610612766               CHA


# Merge team abbreviation and team rank by team id

In [18]:
league_rank_df = pd.merge(team_df, standings_df, left_on='Team_ID', right_on='TeamID', how='inner')
print(league_rank_df)

       Team_ID Team_Abbreviation      TeamID  Rank
0   1610612737               ATL  1610612737    22
1   1610612738               BOS  1610612738     1
2   1610612739               CLE  1610612739     6
3   1610612740               NOP  1610612740    12
4   1610612741               CHI  1610612741    21
5   1610612742               DAL  1610612742    15
6   1610612743               DEN  1610612743     5
7   1610612744               GSW  1610612744    20
8   1610612745               HOU  1610612745    19
9   1610612746               LAC  1610612746     4
10  1610612748               MIA  1610612748    16
11  1610612749               MIL  1610612749     7
12  1610612750               MIN  1610612750     3
13  1610612751               BKN  1610612751    23
14  1610612752               NYK  1610612752     8
15  1610612753               ORL  1610612753    14
16  1610612754               IND  1610612754    13
17  1610612755               PHI  1610612755     9
18  1610612756               PH

In [19]:
league_rank_df = league_rank_df.drop(['Team_ID', 'TeamID'], axis=1)

In [20]:
league_rank_df['TEAM_ABBREVIATION'] = league_rank_df['Team_Abbreviation']
league_rank_df = league_rank_df.drop(['Team_Abbreviation'], axis=1)
print(league_rank_df)

    Rank TEAM_ABBREVIATION
0     22               ATL
1      1               BOS
2      6               CLE
3     12               NOP
4     21               CHI
5     15               DAL
6      5               DEN
7     20               GSW
8     19               HOU
9      4               LAC
10    16               MIA
11     7               MIL
12     3               MIN
13    23               BKN
14     8               NYK
15    14               ORL
16    13               IND
17     9               PHI
18    11               PHX
19    26               POR
20    10               SAC
21    28               SAS
22     2               OKC
23    25               TOR
24    18               UTA
25    24               MEM
26    29               WAS
27    30               DET
28    27               CHA


# Merge league_rank with lal_games

In [21]:
lal_games_ranked = pd.merge(lal_games, league_rank_df[['TEAM_ABBREVIATION', 'Rank']], on='TEAM_ABBREVIATION', how='left')
print(lal_games_ranked)


   WL  FGM  FGA  FG_PCT  FG3M  FG3A  FG3_PCT  FTM  FTA  FT_PCT  ...  DREB  \
0   L   41   90   0.456    10    29    0.345   15   20   0.750  ...    31   
1   W   36   84   0.429     5    29    0.172   23   29   0.793  ...    34   
2   L   43   98   0.439    15    45    0.333   26   34   0.765  ...    45   
3   W   42   79   0.532     8    27    0.296   14   18   0.778  ...    37   
4   W   46   90   0.511    11    32    0.344   27   36   0.750  ...    42   
5   L   37   84   0.440     8    30    0.267   19   21   0.905  ...    30   
6   L   44   81   0.543     8    26    0.308   11   14   0.786  ...    33   
7   L   36   85   0.424     7    25    0.280   15   24   0.625  ...    29   
8   W   42   84   0.500    12    27    0.444   26   34   0.765  ...    34   
9   W   41   84   0.488     4    19    0.211   30   37   0.811  ...    34   
10  W   47   81   0.580    22    35    0.629   18   24   0.750  ...    44   
11  L   42   83   0.506    11    29    0.379   15   19   0.789  ...    36   

In [22]:
lal_games_ranked.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   WL                 52 non-null     object 
 1   FGM                52 non-null     int64  
 2   FGA                52 non-null     int64  
 3   FG_PCT             52 non-null     float64
 4   FG3M               52 non-null     int64  
 5   FG3A               52 non-null     int64  
 6   FG3_PCT            52 non-null     float64
 7   FTM                52 non-null     int64  
 8   FTA                52 non-null     int64  
 9   FT_PCT             52 non-null     float64
 10  OREB               52 non-null     int64  
 11  DREB               52 non-null     int64  
 12  REB                52 non-null     int64  
 13  AST                52 non-null     int64  
 14  STL                52 non-null     int64  
 15  BLK                52 non-null     int64  
 16  TOV                52 non-nu

In [23]:
lal_games_ranked.head()

,WL,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,TEAM_ABBREVIATION,Rank
0,L,41,90,0.456,10,29,0.345,15,20,0.750,...,31,44,23,5,4,12,18,107,DEN,5
1,W,36,84,0.429,5,29,0.172,23,29,0.793,...,34,42,24,14,7,16,23,100,PHX,11
2,L,43,98,0.439,15,45,0.333,26,34,0.765,...,45,53,27,4,4,12,22,127,SAC,10
3,W,42,79,0.532,8,27,0.296,14,18,0.778,...,37,40,28,7,10,16,17,106,ORL,14
4,W,46,90,0.511,11,32,0.344,27,36,0.750,...,42,51,28,10,9,14,24,130,LAC,4


In [24]:
lal_games_ranked.tail()

,WL,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,TEAM_ABBREVIATION,Rank
47,L,45,91,0.495,13,35,0.371,16,26,0.615,...,31,35,26,10,6,12,13,119,HOU,19
48,L,48,90,0.533,12,33,0.364,14,25,0.560,...,30,37,28,12,3,14,16,122,ATL,22
49,W,38,94,0.404,19,36,0.528,19,26,0.731,...,37,52,30,11,2,7,8,114,BOS,1
50,W,39,80,0.488,12,31,0.387,23,27,0.852,...,35,38,28,4,11,5,16,113,NYK,8
51,W,50,89,0.562,8,34,0.235,16,19,0.842,...,41,48,36,5,6,17,12,124,CHA,27


We can see now that the team has the right ranking so it worked as planned

In [25]:
lal_games_ranked = lal_games_ranked.drop(['TEAM_ABBREVIATION'], axis=1)

Now we have our training dataset ready to go, now it is time to set up the prediction dataset for future predictions

# Get Average Lakers Game

In [26]:
from nba_api.stats.endpoints import TeamYearByYearStats

# Specify the team ID for the desired team
team_id = '1610612747'  # Team ID for LAL is '1610612747'

# Create an instance of the TeamYearByYearStats endpoint
team_stats = TeamYearByYearStats(team_id)

# Retrieve the data from the API
team_stats_data = team_stats.get_data_frames()[0]
lal_team_stats = pd.DataFrame(team_stats_data)
lal_team_stats_2024 = lal_team_stats[lal_team_stats['YEAR'] == '2023-24']

In [27]:
lal_team_stats_2024.head()

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
75,1610612747,Los Angeles,Lakers,2023-24,52,27,25,0.519,9,4,...,432,1804,2236,1471,854,409,731,286,6044,4


In [28]:
lal_team_stats_2024.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 75 to 75
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   TEAM_ID                1 non-null      int64  
 1   TEAM_CITY              1 non-null      object 
 2   TEAM_NAME              1 non-null      object 
 3   YEAR                   1 non-null      object 
 4   GP                     1 non-null      int64  
 5   WINS                   1 non-null      int64  
 6   LOSSES                 1 non-null      int64  
 7   WIN_PCT                1 non-null      float64
 8   CONF_RANK              1 non-null      int64  
 9   DIV_RANK               1 non-null      int64  
 10  PO_WINS                1 non-null      int64  
 11  PO_LOSSES              1 non-null      int64  
 12  CONF_COUNT             1 non-null      float64
 13  DIV_COUNT              1 non-null      int64  
 14  NBA_FINALS_APPEARANCE  1 non-null      object 
 15  FGM          

find out what pts rank is: rank of pts per game

Now we make the pipleline to run the season avg stats for the Lakers, so I can use the new averages for each game prediction

In [29]:
from sklearn.base import BaseEstimator, TransformerMixin
class InitialDropper(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.drop(['TEAM_ID', 'TEAM_CITY','YEAR','GP', 'TEAM_NAME', 'WINS', 'LOSSES', 'WIN_PCT', 'CONF_RANK', 'DIV_RANK', 'PO_WINS', 'PO_LOSSES','CONF_COUNT','DIV_COUNT','NBA_FINALS_APPEARANCE', 'PTS_RANK'], axis=1)
    
    

In [30]:
class Conversion(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X['FGM'] = X['FGM'] / len(lal_games_ranked)
        X['FGA'] = X['FGA'] / len(lal_games_ranked)
        X['FG3M'] = X['FG3M'] / len(lal_games_ranked)
        X['FG3A'] = X['FG3A'] / len(lal_games_ranked)
        X['FTM'] = X['FTM'] / len(lal_games_ranked)
        X['FTA'] = X['FTA'] / len(lal_games_ranked)
        X['OREB'] = X['OREB'] / len(lal_games_ranked)
        X['DREB'] = X['DREB'] / len(lal_games_ranked)
        X['REB'] = X['REB'] / len(lal_games_ranked)
        X['AST'] = X['AST'] / len(lal_games_ranked)
        X['STL'] = X['STL'] / len(lal_games_ranked)
        X['BLK'] = X['BLK'] / len(lal_games_ranked)
        X['TOV'] = X['TOV'] / len(lal_games_ranked)
        X['PF'] = X['PF'] / len(lal_games_ranked)
        X['PTS'] = X['PTS'] / len(lal_games_ranked)

        return X

In [31]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([("initialdropper", InitialDropper()),
                     ("conversion", Conversion())])

In [32]:
avg_lal_game = pipeline.fit_transform(lal_team_stats_2024)

In [33]:
avg_lal_game

,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS
75,43.0,87.461538,0.492,11.269231,30.730769,0.367,18.961538,24.557692,0.772,8.307692,34.692308,43.0,28.288462,16.423077,7.865385,14.057692,5.5,116.230769


# Train Test Split & Scaler

In [34]:
%%script echo skipping
from sklearn.model_selection import train_test_split

X = lal_games_ranked.drop(['WL'], axis=1)
y = lal_games_ranked['WL']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

Couldn't find program: 'echo'


In [35]:
%%script echo skipping
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

Couldn't find program: 'echo'


# Model Training & Score

In [36]:
%%script echo skipping
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

clf.fit(X_train, y_train)

Couldn't find program: 'echo'


In [37]:
%%script echo skipping
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Couldn't find program: 'echo'


# Train with Whole Season

In [38]:
%%script echo skipping
X_train = lal_games_ranked.drop(['WL'], axis=1)
y_train = lal_games_ranked['WL']

X_test = avg_lal_game

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

Couldn't find program: 'echo'


In [39]:
%%script echo skipping
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

Couldn't find program: 'echo'


# Game Prediction

In [40]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

opponent = 'CHA'
avg_lal_game['Rank'] = league_rank_df.loc[league_rank_df['TEAM_ABBREVIATION'] == opponent, 'Rank'].iloc[0]

X_test = avg_lal_game
X_test = scaler.fit_transform(X_test)
print("Opponent Rank:", avg_lal_game['Rank'].to_string(index=False))

Opponent Rank: 27


In [41]:
import pickle
Lakers_Predictor = 'Lakers_Predictor.pkl'
clf = pickle.load(open(Lakers_Predictor, 'rb'))

In [42]:
prediction = clf.predict(X_test)

In [43]:
if prediction[0] == 'W':
    print("The Lakers Will Win!")
elif prediction[0] == 'L':
    print("The Lakers Will Lose. :(")
else:
    print("ERROR!")


The Lakers Will Win!


# Save the Model

In [44]:
model = clf
Lakers_Predictor = 'Lakers_Predictor.pkl'
pickle.dump(model, open(Lakers_Predictor, 'wb'))